<a href="https://colab.research.google.com/github/HesusG/diagnostico-lineas-accion/blob/main/Semana2/notebooks/03_regresion_correlacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión Lineal y Correlación

## Objetivos
- Calcular e interpretar coeficientes de correlación
- Crear modelos de regresión lineal simple
- Interpretar R², pendiente e intercepto
- Verificar supuestos de regresión
- Hacer predicciones con modelos
- Construir modelos de regresión múltiple

---

## 1. Preparación

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
%matplotlib inline

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/HesusG/diagnostico-lineas-accion/main/Semana1/datos/ejemplo_satisfaccion_clientes.csv')
print(f"Dataset cargado: {df.shape[0]} registros, {df.shape[1]} variables")
df.head()

## 2. Matriz de Correlación

Primero, exploramos las correlaciones entre **todas** las variables numéricas.

In [ ]:
# Seleccionar variables numéricas
vars_numericas = ['edad', 'tiempo_servicio', 'satisfaccion', 'calidad_atencion', 'tiempo_espera']
df_numerico = df[vars_numericas]

# Calcular matriz de correlación
correlacion = df_numerico.corr()

print("MATRIZ DE CORRELACIÓN (Pearson)")
print("="*60)
print(correlacion.round(3))

In [ ]:
# Heatmap de correlaciones
plt.figure(figsize=(10, 8))
sns.heatmap(correlacion, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            fmt='.3f', vmin=-1, vmax=1)
plt.title('Matriz de Correlación - Variables Numéricas', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Identificar correlaciones fuertes con satisfacción
print("\nCorrelaciones con SATISFACCIÓN:")
print("="*60)
corr_satisfaccion = correlacion['satisfaccion'].sort_values(ascending=False)

for var, corr_val in corr_satisfaccion.items():
    if var != 'satisfaccion':
        if abs(corr_val) >= 0.7:
            fuerza = "FUERTE"
        elif abs(corr_val) >= 0.3:
            fuerza = "MODERADA"
        else:
            fuerza = "DÉBIL"
        
        direccion = "positiva" if corr_val > 0 else "negativa"
        print(f"{var:20s}: r = {corr_val:7.3f} → Correlación {fuerza} {direccion}")

## 3. Correlación de Pearson: Análisis Detallado

### Pregunta de investigación:
> ¿Existe relación lineal entre el tiempo de espera y la satisfacción?

In [ ]:
# Calcular correlación con prueba de significancia
r, p_value = stats.pearsonr(df['tiempo_espera'], df['satisfaccion'])

print("="*60)
print("CORRELACIÓN: TIEMPO DE ESPERA vs SATISFACCIÓN")
print("="*60)
print(f"Coeficiente de Pearson (r): {r:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"R² (varianza compartida): {r**2:.4f} ({r**2*100:.1f}%)")

# Interpretación
if abs(r) < 0.3:
    fuerza = "DÉBIL"
elif abs(r) < 0.7:
    fuerza = "MODERADA"
else:
    fuerza = "FUERTE"

direccion = "POSITIVA" if r > 0 else "NEGATIVA"

print(f"\n💡 Correlación {fuerza} {direccion}")

if p_value < 0.05:
    print("✗ La correlación es estadísticamente SIGNIFICATIVA")
else:
    print("✓ La correlación NO es estadísticamente significativa")

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['tiempo_espera'], df['satisfaccion'], alpha=0.6, edgecolor='black', s=60)
plt.xlabel('Tiempo de Espera (minutos)', fontsize=12)
plt.ylabel('Satisfacción (1-10)', fontsize=12)
plt.title(f'Relación: Tiempo de Espera vs Satisfacción (r={r:.3f}, p={p_value:.4f})',
          fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Regresión Lineal Simple

### Modelo:
$$\text{Satisfacción} = \beta_0 + \beta_1 \times \text{Tiempo de Espera} + \epsilon$$

Donde:
- **β₀**: Intercepto (satisfacción cuando tiempo_espera = 0)
- **β₁**: Pendiente (cambio en satisfacción por cada minuto adicional)
- **ε**: Error (residuo)

In [ ]:
# Opción 1: Regresión con scipy (método rápido)
pendiente, intercepto, r_value, p_value_reg, std_err = stats.linregress(
    df['tiempo_espera'], df['satisfaccion']
)

print("="*60)
print("REGRESIÓN LINEAL SIMPLE (scipy)")
print("="*60)
print(f"Ecuación: Satisfacción = {intercepto:.4f} + ({pendiente:.4f}) × Tiempo_Espera")
print(f"\nCoeficientes:")
print(f"  Intercepto (β₀): {intercepto:.4f}")
print(f"  Pendiente (β₁):  {pendiente:.4f}")
print(f"\nBondad de ajuste:")
print(f"  R²: {r_value**2:.4f}")
print(f"  p-value: {p_value_reg:.4f}")
print(f"  Error estándar: {std_err:.4f}")

In [ ]:
# Opción 2: Regresión con statsmodels (más completo)
X = df['tiempo_espera']
X = sm.add_constant(X)  # Añadir intercepto
Y = df['satisfaccion']

modelo = sm.OLS(Y, X).fit()

print("\n" + "="*60)
print("RESUMEN COMPLETO DEL MODELO (statsmodels)")
print("="*60)
print(modelo.summary())

## 5. Interpretación de Resultados

In [ ]:
print("="*70)
print("INTERPRETACIÓN EN CONTEXTO DE NEGOCIO")
print("="*70)

print(f"\n1. INTERCEPTO (β₀ = {intercepto:.2f}):")
print(f"   → Cuando el tiempo de espera es 0 minutos, la satisfacción esperada es {intercepto:.2f}")
print(f"   (Este valor puede ser teórico si tiempo_espera=0 está fuera del rango observado)")

print(f"\n2. PENDIENTE (β₁ = {pendiente:.4f}):")
if pendiente < 0:
    print(f"   → Por cada MINUTO ADICIONAL de espera, la satisfacción DISMINUYE {abs(pendiente):.4f} puntos")
    print(f"   → Si el tiempo de espera aumenta de 20 a 30 minutos (10 min más):")
    print(f"     Satisfacción baja aproximadamente {abs(pendiente)*10:.2f} puntos")
else:
    print(f"   → Por cada MINUTO ADICIONAL de espera, la satisfacción AUMENTA {pendiente:.4f} puntos")

print(f"\n3. R² = {r_value**2:.4f}:")
print(f"   → El {r_value**2*100:.1f}% de la variabilidad en satisfacción se explica por el tiempo de espera")
print(f"   → El {(1-r_value**2)*100:.1f}% restante se debe a otros factores no incluidos en el modelo")

print(f"\n4. SIGNIFICANCIA ESTADÍSTICA:")
if p_value_reg < 0.001:
    print(f"   → p-value < 0.001: Relación MUY significativa")
elif p_value_reg < 0.05:
    print(f"   → p-value = {p_value_reg:.4f}: Relación significativa")
else:
    print(f"   → p-value = {p_value_reg:.4f}: Relación NO significativa")

## 6. Visualización del Modelo

In [ ]:
# Línea de regresión con intervalo de confianza
plt.figure(figsize=(12, 6))
sns.regplot(x='tiempo_espera', y='satisfaccion', data=df,
            scatter_kws={'alpha':0.5, 'edgecolor':'black', 's':60},
            line_kws={'color':'red', 'linewidth':2.5})

plt.xlabel('Tiempo de Espera (minutos)', fontsize=12)
plt.ylabel('Satisfacción (1-10)', fontsize=12)
plt.title(f'Regresión Lineal: Satisfacción = {intercepto:.2f} + ({pendiente:.4f}) × Tiempo_Espera\n'
          f'R² = {r_value**2:.4f}', fontsize=13, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Predicciones vs Valores Reales
predicciones = intercepto + pendiente * df['tiempo_espera']

plt.figure(figsize=(10, 6))
plt.scatter(df['satisfaccion'], predicciones, alpha=0.6, edgecolor='black', s=60)
plt.plot([df['satisfaccion'].min(), df['satisfaccion'].max()],
         [df['satisfaccion'].min(), df['satisfaccion'].max()],
         'r--', linewidth=2.5, label='Predicción perfecta')
plt.xlabel('Satisfacción Real', fontsize=12)
plt.ylabel('Satisfacción Predicha', fontsize=12)
plt.title('Valores Reales vs Predicciones', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Verificación de Supuestos

### Supuesto 1: Linealidad
La relación entre X e Y debe ser lineal (ya verificamos con scatter plot).

### Supuesto 2: Independencia de Residuos

### Supuesto 3: Homocedasticidad
La varianza de los residuos debe ser constante.

In [ ]:
# Calcular residuos
residuos = df['satisfaccion'] - predicciones

print("Estadísticos de Residuos:")
print(f"Media: {residuos.mean():.6f} (debe ser ~0)")
print(f"Desviación estándar: {residuos.std():.4f}")
print(f"Mínimo: {residuos.min():.4f}")
print(f"Máximo: {residuos.max():.4f}")

In [ ]:
# Gráficos de diagnóstico
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Residuos vs Valores Ajustados
axes[0, 0].scatter(predicciones, residuos, alpha=0.6, edgecolor='black')
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Valores Ajustados')
axes[0, 0].set_ylabel('Residuos')
axes[0, 0].set_title('Residuos vs Valores Ajustados', fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# 2. Q-Q Plot (Normalidad de residuos)
stats.probplot(residuos, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('Q-Q Plot de Residuos', fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# 3. Histograma de Residuos
axes[1, 0].hist(residuos, bins=20, edgecolor='black', alpha=0.7, color='skyblue')
axes[1, 0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Residuos')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].set_title('Distribución de Residuos', fontweight='bold')
axes[1, 0].grid(alpha=0.3, axis='y')

# 4. Scale-Location (Raíz de residuos estandarizados)
residuos_std = (residuos - residuos.mean()) / residuos.std()
axes[1, 1].scatter(predicciones, np.sqrt(np.abs(residuos_std)), alpha=0.6, edgecolor='black')
axes[1, 1].set_xlabel('Valores Ajustados')
axes[1, 1].set_ylabel('√|Residuos Estandarizados|')
axes[1, 1].set_title('Scale-Location', fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Prueba de normalidad de residuos (Shapiro-Wilk)
stat_shapiro, p_shapiro = stats.shapiro(residuos)

print("\nPRUEBA DE NORMALIDAD DE RESIDUOS:")
print(f"Shapiro-Wilk: W={stat_shapiro:.4f}, p={p_shapiro:.4f}")

if p_shapiro > 0.05:
    print("✓ Residuos siguen distribución normal")
else:
    print("⚠️ Residuos NO siguen distribución normal")
    print("   Considerar transformación de variables")

## 8. Hacer Predicciones

In [ ]:
# Rango de datos observados
print("Rango de Tiempo de Espera observado:")
print(f"Mínimo: {df['tiempo_espera'].min()} minutos")
print(f"Máximo: {df['tiempo_espera'].max()} minutos")

print("\n" + "="*60)
print("PREDICCIONES DE EJEMPLOS")
print("="*60)

# Ejemplos de predicción
tiempos_ejemplo = [10, 20, 25, 30, 35]

for tiempo in tiempos_ejemplo:
    sat_pred = intercepto + pendiente * tiempo
    print(f"\nTiempo de espera = {tiempo} min → Satisfacción predicha = {sat_pred:.2f}")
    
    # Advertencia si está fuera del rango
    if tiempo < df['tiempo_espera'].min() or tiempo > df['tiempo_espera'].max():
        print("   ⚠️ Advertencia: Extrapolación (fuera del rango observado)")

In [ ]:
# Intervalos de confianza para predicciones (con statsmodels)
nuevos_tiempos = pd.DataFrame({'tiempo_espera': [15, 25, 35]})
nuevos_tiempos = sm.add_constant(nuevos_tiempos)

predicciones_nuevas = modelo.get_prediction(nuevos_tiempos)
df_pred = predicciones_nuevas.summary_frame(alpha=0.05)

print("\nPredicciones con Intervalos de Confianza 95%:")
df_pred.index = [15, 25, 35]
print(df_pred[['mean', 'mean_ci_lower', 'mean_ci_upper']].round(2))

## 9. Segundo Modelo: Calidad de Atención vs Satisfacción

In [ ]:
# Correlación
r2, p2 = stats.pearsonr(df['calidad_atencion'], df['satisfaccion'])

print("="*60)
print("MODELO 2: CALIDAD DE ATENCIÓN → SATISFACCIÓN")
print("="*60)
print(f"Correlación (r): {r2:.4f}")
print(f"R²: {r2**2:.4f}")
print(f"p-value: {p2:.4f}")

# Regresión
pend2, inter2, r_val2, p_val2, se2 = stats.linregress(df['calidad_atencion'], df['satisfaccion'])

print(f"\nEcuación: Satisfacción = {inter2:.4f} + ({pend2:.4f}) × Calidad_Atención")
print(f"\nInterpretación:")
print(f"  Por cada punto adicional en calidad de atención,")
print(f"  la satisfacción {'aumenta' if pend2 > 0 else 'disminuye'} {abs(pend2):.4f} puntos")

In [ ]:
# Visualización
plt.figure(figsize=(10, 6))
sns.regplot(x='calidad_atencion', y='satisfaccion', data=df,
            scatter_kws={'alpha':0.5, 'edgecolor':'black'},
            line_kws={'color':'green', 'linewidth':2.5})
plt.xlabel('Calidad de Atención (1-10)', fontsize=12)
plt.ylabel('Satisfacción (1-10)', fontsize=12)
plt.title(f'Calidad de Atención vs Satisfacción (R² = {r_val2**2:.4f})',
          fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Regresión Lineal Múltiple

Ahora usamos **múltiples** variables para predecir satisfacción:

$$\text{Satisfacción} = \beta_0 + \beta_1 \times \text{Tiempo\_Espera} + \beta_2 \times \text{Calidad\_Atención} + \beta_3 \times \text{Tiempo\_Servicio} + \epsilon$$

In [ ]:
# Preparar datos
X_multi = df[['tiempo_espera', 'calidad_atencion', 'tiempo_servicio']]
X_multi = sm.add_constant(X_multi)
Y = df['satisfaccion']

# Ajustar modelo
modelo_multi = sm.OLS(Y, X_multi).fit()

print("="*70)
print("REGRESIÓN LINEAL MÚLTIPLE")
print("="*70)
print(modelo_multi.summary())

In [ ]:
# Ecuación del modelo
print("\nECUACIÓN DEL MODELO:")
print("="*70)
print(f"Satisfacción = {modelo_multi.params['const']:.4f}")

for var in ['tiempo_espera', 'calidad_atencion', 'tiempo_servicio']:
    coef = modelo_multi.params[var]
    signo = '+' if coef >= 0 else ''
    print(f"             {signo}{coef:.4f} × {var}")

print(f"\nR² ajustado: {modelo_multi.rsquared_adj:.4f}")
print(f"R² simple:   {modelo_multi.rsquared:.4f}")

In [ ]:
# Interpretación de coeficientes
print("\nINTERPRETACIÓN DE COEFICIENTES:")
print("="*70)

for var in ['tiempo_espera', 'calidad_atencion', 'tiempo_servicio']:
    coef = modelo_multi.params[var]
    p_val = modelo_multi.pvalues[var]
    
    print(f"\n{var}:")
    print(f"  Coeficiente: {coef:.4f}")
    print(f"  p-value: {p_val:.4f}")
    
    if p_val < 0.05:
        print(f"  ✗ Variable SIGNIFICATIVA")
        print(f"  Manteniendo las demás variables constantes, un aumento de 1 unidad")
        print(f"  en {var} {'aumenta' if coef > 0 else 'disminuye'} la satisfacción en {abs(coef):.4f} puntos")
    else:
        print(f"  ✓ Variable NO significativa (puede eliminarse del modelo)")

## 11. Comparación de Modelos

In [ ]:
# Comparar modelos
modelos_comparacion = pd.DataFrame({
    'Modelo': [
        'Simple: Tiempo_Espera',
        'Simple: Calidad_Atención',
        'Múltiple: 3 variables'
    ],
    'R²': [
        r_value**2,
        r_val2**2,
        modelo_multi.rsquared
    ],
    'R² ajustado': [
        np.nan,  # No aplica para regresión simple
        np.nan,
        modelo_multi.rsquared_adj
    ],
    'AIC': [
        np.nan,
        np.nan,
        modelo_multi.aic
    ]
})

print("="*70)
print("COMPARACIÓN DE MODELOS")
print("="*70)
print(modelos_comparacion.to_string(index=False))

print("\n💡 ¿Qué modelo es mejor?")
print(f"   El modelo múltiple explica {modelo_multi.rsquared*100:.1f}% de la varianza")
print(f"   vs {r_value**2*100:.1f}% del mejor modelo simple")
print(f"   → Mejora de {(modelo_multi.rsquared - r_value**2)*100:.1f} puntos porcentuales")

## 12. Predicciones con Modelo Múltiple

In [ ]:
# Ejemplo de predicción
print("EJEMPLO DE PREDICCIÓN CON MODELO MÚLTIPLE")
print("="*70)

# Nuevo caso
nuevo_caso = pd.DataFrame({
    'const': [1],
    'tiempo_espera': [25],
    'calidad_atencion': [8],
    'tiempo_servicio': [12]
})

prediccion = modelo_multi.predict(nuevo_caso)

print(f"\nCaracterísticas del beneficiario:")
print(f"  Tiempo de espera: 25 minutos")
print(f"  Calidad de atención: 8/10")
print(f"  Tiempo de servicio: 12 meses")
print(f"\nSatisfacción predicha: {prediccion[0]:.2f}/10")

## 13. Reporte Ejecutivo

In [ ]:
print("="*70)
print("REPORTE EJECUTIVO - REGRESIÓN Y CORRELACIÓN")
print("="*70)

print("\n🎯 OBJETIVO:")
print("   Identificar factores que predicen la satisfacción del cliente")

print("\n📊 HALLAZGOS PRINCIPALES:")

# Top 3 correlaciones con satisfacción
top_corr = correlacion['satisfaccion'].drop('satisfaccion').abs().sort_values(ascending=False).head(3)
print("\n   Variables más correlacionadas con satisfacción:")
for i, (var, val) in enumerate(top_corr.items(), 1):
    direccion = "positiva" if correlacion.loc[var, 'satisfaccion'] > 0 else "negativa"
    print(f"   {i}. {var}: r = {correlacion.loc[var, 'satisfaccion']:.3f} ({direccion})")

print(f"\n💡 MODELO RECOMENDADO: Regresión Múltiple")
print(f"   R² = {modelo_multi.rsquared:.4f} ({modelo_multi.rsquared*100:.1f}% de varianza explicada)")
print(f"\n   Variables significativas:")

for var in ['tiempo_espera', 'calidad_atencion', 'tiempo_servicio']:
    if modelo_multi.pvalues[var] < 0.05:
        coef = modelo_multi.params[var]
        print(f"   • {var}: β = {coef:.4f} (p < 0.05)")

print("\n📌 RECOMENDACIONES:")
if modelo_multi.params['tiempo_espera'] < 0 and modelo_multi.pvalues['tiempo_espera'] < 0.05:
    print("   1. Reducir tiempos de espera para mejorar satisfacción")
if modelo_multi.params['calidad_atencion'] > 0 and modelo_multi.pvalues['calidad_atencion'] < 0.05:
    print("   2. Invertir en capacitación para mejorar calidad de atención")

print("\n" + "="*70)

## 14. Ejercicios Propuestos

### Ejercicio 1: Edad vs Satisfacción
Crea un modelo de regresión simple entre edad y satisfacción. ¿Es significativo?

### Ejercicio 2: Modelo Reducido
Elimina las variables NO significativas del modelo múltiple y vuelve a ajustar. ¿Mejora el R² ajustado?

### Ejercicio 3: Residuos
Identifica observaciones con residuos grandes (>2 SD). ¿Son outliers?

### Ejercicio 4: Interacciones
Crea un modelo con término de interacción: `tiempo_espera * calidad_atencion`


In [ ]:
# Tu código aquí


---

## Resumen

En este notebook aprendiste a:
- ✓ Calcular e interpretar correlaciones de Pearson
- ✓ Crear modelos de regresión lineal simple
- ✓ Interpretar coeficientes (intercepto, pendiente, R²)
- ✓ Verificar supuestos de regresión (linealidad, normalidad, homocedasticidad)
- ✓ Hacer predicciones con modelos
- ✓ Construir modelos de regresión múltiple
- ✓ Comparar modelos y seleccionar el mejor
- ✓ Interpretar resultados en contexto de negocio

**¡Felicidades!** Has completado el Módulo 1 de Estadística.

**Siguiente módulo:** Ética, Compromiso Social y Diagnóstico Estratégico (Semana 3)
